# get Cause - efffect phrase from the sentence.

In [1]:
#filtered_rows.reset_index(drop=True, inplace=True)


## 1. load cause-effect data

In [2]:
#load CE semeval dataset
import pandas as pd
sem_CE_test= pd.read_csv('data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv')
pm_DB = sem_CE_test
pm_DB

,Unnamed: 0,sentence
0,0,Using solar electricity instead of conventiona...
1,1,"Thus, evaluating capital punishment as a form ..."
2,2,Sudden death from inhalation of petroleum ...
3,3,Stimulation of the posterior horn of the med...
4,4,Sip the tea slowly to reduce stomach pain fr...
...,...,...
186,186,The disruption has been caused by work bei...
187,187,"Most illnesses, including colds and flu , c..."
188,188,Every child and young person seen at Starship ...
189,189,The movie gives the inaccurate impression ...


## 2. load few shot database

In [3]:
#load semeval sentence and patterns.
semeval_data = pd.read_csv('data/in/RAG/comb_SCITEsemADE_CausalityPattern.csv')# 

semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

## 3. identify cause- effect phrase

In [4]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [5]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')

#+ fewshot_examples
def C_E_CP_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-4o",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify all the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause1": "", "effect1": "", "causality_phrase1": "", "cause2": "", "effect2": "", "causality_phrase2": "",
          "cause3": "", "effect3": "", "causality_phrase3": "", ....}. Make sure your JSON ouput is closed with } at the end.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
          
          Rule 3: There could be multiple cause-effect phrases in the sentence. You should extract all 
          of them. For instance, consider the input sentence "Mutations in MCFD2 causing F5F8D are null
          mutations but also missense mutations, which cause destabilization of its flexible structure
          and the loss of its interaction with ERGIC-53 .". 
          Your identified output will be:
          {"cause1": "Mutations in MCFD2", "effect1": "F5F8D", "causality_phrase1": "causing", 
          "cause2": "missense mutations", "effect2": "destabilization of its flexible structure", "causality_phrase2": "cause",
          "cause3": "missense mutations", "effect3": "the loss of its interaction with ERGIC-53", "causality_phrase3": "cause"

          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=200,
      top_p=1
    )
    
    return response.choices[0].message.content


In [6]:
def get_fewshotexamples_C_E_CP(fewshot_index):

    
    #fewshot_index = [1, 5, 8, 10, 30, 100, 102, 105, 109, 120]
    
    
    train_sentence = semeval_data['sentence'].to_list()
    train_causalityPhrase = semeval_data['causality_phrase'].to_list()

    fewshot_example_sentence = [train_sentence[i] for i in fewshot_index]
    fewshot_example_causalityPhrase = [train_causalityPhrase[i] for i in fewshot_index]


    import json
    #fewshot_examples = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    fewshot_examples = {f'example {i + 1}': {'sentence': sentence, 'causalityPhrase': fewshot_example_causalityPhrase[i]} 
                    for i, sentence in enumerate(fewshot_example_sentence)}
    
    # Assuming json_data is already defined
    fewshot_examples = json.dumps(fewshot_examples)
    #print(fewshot_examples)
    
    return fewshot_examples

In [7]:
# semeval
filtered_rows = pm_DB

In [8]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        #print('ind :', ind)
        window_size = len(p)

        # Slide through input_sentence with a window size 
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('causality phrase: ',p, 'index is', ind)
                #print(f"Window '{window}' - LS: {score}")

    #print('fewshot_index', fewshot_index)
    fewshot_index = list(set(fewshot_index))
    #print('fewshot_index', fewshot_index)
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [9]:
import json
import re

def close_json_string(json_string):
    
    
    # Remove entries where "cause", "effect", and "causality_phrase" are all empty
    pattern = re.compile(r'"cause\d+":\s*"",\s*"effect\d+":\s*"",\s*"causality_phrase\d+":\s*""\s*,?\s*')
    json_string = re.sub(pattern, '', json_string)
    
    # Remove incomplete index entries
    incomplete_index_pattern = re.compile(r'"cause\d+":\s*""\s*,?\s*')
    json_string = re.sub(incomplete_index_pattern, '', json_string)
    
    # Remove trailing comma if exists
    json_string = re.sub(r',\s*}$', '}', json_string.strip())
    json_string = re.sub(r',\s*$', '', json_string.strip())
    
    
    
    # Find the last complete index
    last_index = re.findall(r'"cause\d+":\s*"[^"]*",\s*"effect\d+":\s*"[^"]*",\s*"causality_phrase\d+":\s*"[^"]*"', json_string)
    
    # If found, truncate the JSON string after the last complete index
    if last_index:
        last_index_end_pos = json_string.rfind(last_index[-1]) + len(last_index[-1])
        json_string = json_string[:last_index_end_pos]
    
    # Count the number of opening and closing braces
    open_braces = json_string.count('{')
    close_braces = json_string.count('}')
    
    # Add missing closing braces if needed
    if open_braces > close_braces:
        json_string += '}' * (open_braces - close_braces)
    
    return json_string

In [10]:
import json
sent_cause_effect_list = []
causality_phrase_list = []

fewshotfound_inputsentence_index=[]
for i in range(len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    # few shot examples augmentation.
    fewshot_index=[]
    fewshot_index = get_fewshot_examples_index(input_sentence)#########FEWSHOT
    #print('fewshot_index before :', fewshot_index)
    ##
    if len(fewshot_index) == 0:
        fewshot_index = [k for k in range(10)]
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
    else:
        fewshotfound_inputsentence_index.append(i)
        
        #fewshot_index = [k for k in range(10)]###############################RANDOMSHOT/zero
        fewshot_examples =  get_fewshotexamples_C_E_CP(fewshot_index) 
        
    ##
    print('fewshot_index :', fewshot_index)
    ##############print('fewshot_examples', fewshot_examples)

    #fewshot_examples = get_fewshotexamples_C_E_CP(fewshot_index) 
    
    #print('fewshotfound_inputsentence_index', fewshotfound_inputsentence_index)
    
    # output generation
    out = C_E_CP_Identifier(input_sentence, fewshot_examples)
    out = close_json_string(out)
    print('out : ', out)

    
    # Attempt to parse the JSON string into a dictionary
    try:
        data = json.loads(out)
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue  # Skip to the next iteration if there is a JSON error

    # Parse JSON string into a dictionary
    data = json.loads(out)

    # Initialize lists
    sent_cause_effect = []
    causality_phrase_row = []

    # Iterate through the keys and collect values
    i_out = 1
    while f'cause{i_out}' in data and f'effect{i_out}' in data and f'causality_phrase{i_out}' in data:
        sent_cause_effect.append((input_sentence, data[f'cause{i_out}'], data[f'effect{i_out}']))
        causality_phrase_row.append(data[f'causality_phrase{i_out}'])
        i_out += 1

    # Print the results
    print("sent_cause_effect:", sent_cause_effect)
    #print("causality_phrase_row:", causality_phrase_row)
    sent_cause_effect_list.append(sent_cause_effect)
    causality_phrase_list.append(causality_phrase_row)
    print('i:', i)
    
    #if i==3:
        #break;
    

fewshot_index : [516, 2005, 1506, 2017, 519, 514, 523, 2087, 1513, 2085]
out :  {"cause1": "solar electricity", "effect1": "pollution", "causality_phrase1": "instead of conventional electricity sources helps to reduce", "cause2": "pollution", "effect2": "smog", "causality_phrase2": "that causes", "cause3": "pollution", "effect3": "acid rain", "causality_phrase3": "that causes", "cause4": "pollution", "effect4": "respiratory illness", "causality_phrase4": "that causes", "cause5": "pollution", "effect5": "global warming", "causality_phrase5": "that causes"}
sent_cause_effect: [('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'solar electricity', 'pollution'), ('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'pollution', 'smog'), ('Using

fewshot_index : [279, 2091, 2089, 2092, 996, 2090, 2093, 2086, 992, 994]
out :  {"cause1": "wine", "effect1": "headache", "causality_phrase1": "from"}
sent_cause_effect: [('I too, get  a headache  from  wine , and was always told that it was the sulfites.', 'wine', 'headache')]
i: 14
fewshot_index : [126, 1532, 468, 1242, 121, 1528, 1527, 1136, 119, 1533]
out :  {"cause": "an intense discussion", "effect": "The accolade", "causality_phrase": "was decided upon"}
sent_cause_effect: []
i: 15
fewshot_index : [593, 1123, 1264, 550, 548, 477, 552, 1261, 521, 517]
out :  {"cause1": "mudslides, tremors, subsidence, superficial or underground water", "effect1": "damages", "causality_phrase1": "caused by", "cause2": "swelling clay soils", "effect2": "verified", "causality_phrase2": "as well as"}
sent_cause_effect: [(' The damages  caused by  mudslides ,  tremors ,  subsidence ,  superficial or underground water  were verified, as well as  swelling clay soils .', 'mudslides, tremors, subsidence, 

fewshot_index : [2120, 590, 1507, 1262, 480, 589, 1532, 520, 1264, 517]
out :  {"cause1": "nitrite", "effect1": "increased reproductive toxicity of landfill leachate after degradation", "causality_phrase1": "was caused by"}
sent_cause_effect: [(' Increased reproductive toxicity  of landfill leachate after degradation was caused by  nitrite .', 'nitrite', 'increased reproductive toxicity of landfill leachate after degradation')]
i: 33
fewshot_index : [1136, 1301, 1643, 2091, 478, 1638, 482, 1631, 1630, 2090]
out :  {"cause1": "440 nuclear reactors", "effect1": "electricity", "causality_phrase1": "is produced by"}
sent_cause_effect: [(' The electricity  is produced by  440 nuclear reactors  in 31 countries.', '440 nuclear reactors', 'electricity')]
i: 34
fewshot_index : [552, 479, 1529, 1533, 591, 518, 593, 549, 555, 477]
out :  {"cause": "salmonella bacteria", "effect": "foodborne illness", "causality_phrase": "caused by"}
sent_cause_effect: []
i: 35
fewshot_index : [324, 479, 482, 554,

out :  {"cause": "Individual donors", "effect": "the suffering", "causality_phrase": "caused by"}
sent_cause_effect: []
i: 52
fewshot_index : [1242, 1510, 2088, 992, 994, 1253, 2089, 1530, 997, 2087]
out :  {"cause": "animal factories", "effect": "parts of the world's oceans", "causality_phrase": "is also destroying"}
sent_cause_effect: []
i: 53
fewshot_index : [894, 993, 1511, 1528, 1527, 892, 1508, 1509, 1515, 994]
out :  {"cause1": "the Florida contingent", "effect1": "Discussion", "causality_phrase1": "ensued from"}
sent_cause_effect: [(' Discussion  ensued from  the Florida contingent  on the fact that very stringent landowner protection laws in Florida make it imperative that the highest supportable appraised vallue be offered first.', 'the Florida contingent', 'Discussion')]
i: 54
fewshot_index : [2088, 2087, 1533, 1241, 997, 1528, 994, 2090, 1242, 2089]
out :  {"cause1": "exertion", "effect1": "satisfaction", "causality_phrase1": "from"}
sent_cause_effect: [('I have always draw

fewshot_index : [549, 516, 551, 550, 1551, 514, 480, 596, 482, 481]
out :  {"cause": "the Earth's atmosphere", "effect": "drag", "causality_phrase": "caused by"}
sent_cause_effect: []
i: 73
fewshot_index : [1529, 1527, 515, 522, 1533, 521, 278, 2089, 2093, 1532]
out :  {"cause1": "germs", "effect1": "sickness", "causality_phrase1": "cause", "cause2": "microbes", "effect2": "sickness", "causality_phrase2": "cause"}
sent_cause_effect: [('The presentation uses animation to show how  germs  and  microbes  cause  sickness , and outlines simple preventive measures.', 'germs', 'sickness'), ('The presentation uses animation to show how  germs  and  microbes  cause  sickness , and outlines simple preventive measures.', 'microbes', 'sickness')]
i: 74
fewshot_index : [522, 1532, 515, 523, 1512, 516, 590, 319, 517, 477]
out :  {"cause": "the discharge of accumulated along geologic faults", "effect": "earthquakes", "causality_phrase": "are caused by"}
sent_cause_effect: []
i: 75
fewshot_index : [12

...

fewshot_index : [1530, 1515, 1819, 1818, 1136, 1514, 1531, 1815, 1512, 1814]
out :  {"cause1": "Four of the entrapments", "effect1": "suffocation", "causality_phrase1": "resulted in"}
sent_cause_effect: [('Four of  the entrapments  resulted in  suffocation : a 7-month-old in Gouverneur, N.Y.; a 7-month-old in New Iberia, La.; a 6-month-old in Summersville, W.Va.; and a 9-month-old in Bronx, N.Y.', 'Four of the entrapments', 'suffocation')]
i: 109
fewshot_index : [1818, 1514, 1508, 1510, 1511, 278, 1813, 1814, 1815, 1242]
out :  {"cause1": "alleged abuse", "effect1": "bruises", "causality_phrase1": "resulted in", "cause2": "alleged abuse", "effect2": "swelling of the brain", "causality_phrase2": "resulted in"}
sent_cause_effect: [(' The alleged abuse  resulted in  bruises  and  swelling  of the brain.', 'alleged abuse', 'bruises'), (' The alleged abuse  resulted in  bruises  and  swelling  of the brain.', 'alleged abuse', 'swelling of the brain')]
i: 110
fewshot_index : [1242, 2004, 201

out :  {"cause": "animals", "effect": "much joy", "causality_phrase": "from"}
sent_cause_effect: []
i: 131
fewshot_index : [280, 518, 614, 612, 1242, 521, 617, 517, 515, 2153]
out :  {"cause1": "acne", "effect1": "stress", "causality_phrase1": "causes", "cause2": "stress", "effect2": "acne breakouts", "causality_phrase2": "triggers"}
sent_cause_effect: [('Like a catch-22 the worst part is that  acne  causes  stress  and  stress  triggers  acne breakouts .', 'acne', 'stress'), ('Like a catch-22 the worst part is that  acne  causes  stress  and  stress  triggers  acne breakouts .', 'stress', 'acne breakouts')]
i: 132
fewshot_index : [1241, 1258, 554, 1533, 1551, 515, 520, 598, 517, 548]
out :  {"cause1": "a bacteria", "effect1": "Salmonellosis", "causality_phrase1": "caused by"}
sent_cause_effect: [('Information about Salmonellosis,  an illness  caused by  a bacteria  found in raw food, soil, or water.', 'a bacteria', 'Salmonellosis')]
i: 133
fewshot_index : [280, 741, 736, 482, 279, 740

fewshot_index : [2009, 1242, 278, 280, 518, 556, 2011, 554, 519, 523]
out :  {"cause1": "virus", "effect1": "1918 flu pandemic", "causality_phrase1": "that caused", "cause2": "virus", "effect2": "young adults and school children", "causality_phrase2": "affects"}
sent_cause_effect: [('Much like  the virus  that caused  the 1918 flu pandemic , this new H1N1 flu virus affects young adults and school children in greater numbers.', 'virus', '1918 flu pandemic'), ('Much like  the virus  that caused  the 1918 flu pandemic , this new H1N1 flu virus affects young adults and school children in greater numbers.', 'virus', 'young adults and school children')]
i: 148
fewshot_index : [1531, 593, 318, 556, 1533, 514, 522, 1527, 321, 1551]
out :  {"cause1": "a hormone called progesterone", "effect1": "changes now seen in the endometrium", "causality_phrase1": "caused by"}
sent_cause_effect: [(' The changes  now seen in the endometrium are caused by  a hormone  called progesterone.', 'a hormone called 

fewshot_index : [521, 279, 2090, 555, 520, 593, 2086, 549, 1506, 517]
out :  {"cause1": "the triggering", "effect1": "a market spike", "causality_phrase1": "caused by", "cause2": "the election", "effect2": "a market spike", "causality_phrase2": "caused by"}
sent_cause_effect: [('A method of mitigating the effect of  a market spike  caused by  the triggering  and  the election  of a conditional order includes monitoring conditional orders.', 'the triggering', 'a market spike'), ('A method of mitigating the effect of  a market spike  caused by  the triggering  and  the election  of a conditional order includes monitoring conditional orders.', 'the election', 'a market spike')]
i: 170
fewshot_index : [1528, 557, 1531, 552, 481, 553, 1530, 2092, 595, 554]
out :  {
  "cause1": "He created and advocated \"flower power,\"",
  "effect1": "a strategy in which antiwar demonstrators promoted positive values like peace and love to dramatize their opposition to",
  "causality_phrase1": "in which",


fewshot_index : [278, 520, 2091, 2094, 2086, 2087, 514, 517, 2088, 523]
out :  {"cause1": "Snails", "effect1": "damage to seedlings, flowers, vegetables and shrubs", "causality_phrase1": "cause", "cause2": "slugs", "effect2": "damage to seedlings, flowers, vegetables and shrubs", "causality_phrase2": "cause"}
sent_cause_effect: [(' Snails  and  slugs  cause  damage  to seedlings, flowers, vegetables and shrubs.', 'Snails', 'damage to seedlings, flowers, vegetables and shrubs'), (' Snails  and  slugs  cause  damage  to seedlings, flowers, vegetables and shrubs.', 'slugs', 'damage to seedlings, flowers, vegetables and shrubs')]
i: 185
fewshot_index : [554, 596, 2087, 555, 1551, 517, 2094, 598, 2093, 591]
out :  {"cause1": "work", "effect1": "disruption", "causality_phrase1": "being carried out by"}
sent_cause_effect: [(' The disruption  has been caused by  work  being carried out by track operator Network Rail.', 'work', 'disruption')]
i: 186
fewshot_index : [1164, 2094, 521, 520, 2091, 

In [11]:
len(filtered_rows)

191

In [12]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

191

In [13]:
filtered_rows['sent_cause_effect'] = sent_cause_effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0,sentence,sent_cause_effect,causality_phrase
0,0,Using solar electricity instead of conventiona...,[(Using solar electricity instead of conventio...,[instead of conventional electricity sources h...
1,1,"Thus, evaluating capital punishment as a form ...","[(Thus, evaluating capital punishment as a for...","[reduced by Sellin to merely, that result in]"
2,2,Sudden death from inhalation of petroleum ...,[( Sudden death from inhalation of petroleu...,[from]
3,3,Stimulation of the posterior horn of the med...,[],[]
4,4,Sip the tea slowly to reduce stomach pain fr...,[(Sip the tea slowly to reduce stomach pain ...,"[from, from, from]"
...,...,...,...,...
186,186,The disruption has been caused by work bei...,[( The disruption has been caused by work b...,[being carried out by]
187,187,"Most illnesses, including colds and flu , c...","[(Most illnesses, including colds and flu ,...","[including, including, cause, cause]"
188,188,Every child and young person seen at Starship ...,[(Every child and young person seen at Starshi...,[had a social work assessment before leaving t...
189,189,The movie gives the inaccurate impression ...,[],[]


In [14]:
#SCITE

pm_DB = pd.read_csv('data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUTkeys.csv')
pm_DB['keys'] = pm_DB['keys'].apply(eval)



C_t = pm_DB['keys'].to_list()


C_p = sent_cause_effect_list

In [15]:
len(sent_cause_effect_list)

191

In [16]:
C_p = [tup for sublist in C_p for tup in sublist]
len(C_p[1])


3

In [17]:
C_p[1]

('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .',
 'pollution',
 'smog')

In [18]:
C_p[0]

('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .',
 'solar electricity',
 'pollution')

In [19]:
len(C_t)

296

In [20]:
unique_C_p = list(set(C_t))
len(unique_C_p)

296

In [21]:
# Remove duplicates while preserving order
seen = set()
unique_C_p = []
for tup in C_t:
    if tup not in seen:
        unique_C_p.append(tup)
        seen.add(tup)


In [22]:
C_p[41]

('Our country is resolving  the war  and  poverty  caused by  the previous administration .',
 'previous administration',
 'war')

In [23]:
C_p[39]

(' Landslides  caused the majority of  the deaths .',
 'Landslides',
 'the majority of the deaths')

In [24]:
unique_C_p[41]

(' Fog ,  rain ,  darkness , and or  blowing snow  lead to  disorientation .',
 'Fog',
 'disorientation')

In [25]:
unique_C_p[1]

('Cutler ended up with  a bleeding stomach ulcer  caused by  the stress  and  hard work  supervising their tours twenty-four hours a day.',
 'hard work',
 'a bleeding stomach ulcer')

In [26]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [27]:
len(C_p)

223

In [28]:
prediction= []
pred_label = []
for i in range(len(C_p)):
    #print(i)
    labeled = 0   
    if (C_p[i] != []):
        if len(C_p[i]) == 3:           
            
            sent = C_p[i][0]
            ####print('sent: ',sent )
            cause_pred = C_p[i][1]
            effect_pred = C_p[i][2]
            
            for i_t in range(len(C_t)):
                #get index of all tuples which match the sent and create sent_found = []
                # List to store indices
                sent_found = []

                # Check if sent is in the C_t at index 0
                for index, (sentence, _, _) in enumerate(C_t):
                    if sentence == sent:
                        sent_found.append(index)
            ####print('sent_found: ', sent_found)
            
            
            #iterate through sent_found and pass each cause_t and effect_t to get score:
            count=0
            for ind in sent_found:
                cause_t = C_t[ind][1]
                effect_t = C_t[ind][2]
                ##print('cause_t: ', cause_t)
                ##print('effect_t: ', effect_t)
                ##print('cause_pred: ', cause_pred)
                ##print('effect_pred: ', effect_pred)
                pred_c=0
                pred_e=0
                
                score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
                    #score = fuzz.ratio(cause_pred, cause_t)################medcause
                if score >90:################90 non -medcause
                    pred_c = 1
                else:
                    pred_c=0
                    


                score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
                    #score = fuzz.ratio(effect_pred, effect_t)################medcause

                if score >90: ################90 non -medcause
                    pred_e = 1
                else:
                    pred_e=0



                if ((pred_e ==1) & (pred_c ==  1)):
                    prediction.append(1)
                    
                    labeled = 1
                    #pred_label.append(labeled)
                    #print('out:', 1)
                    count=count+1
                    if count>1:
                        print('Cp',C_p[i] )
                        print('Ct', C_t[ind])
    
    
    
    
    pred_label.append(labeled)
                    
    print(labeled, i)



                
    

 

prediction length lower pollution p: respiratory illness
prediction length lower pollution p: global warming
0 0
prediction length lower smog p: acid rain
prediction length lower smog p: respiratory illness
prediction length lower smog p: global warming
1 1
prediction length lower acid rain p: respiratory illness
prediction length lower acid rain p: global warming
1 2
1 3
prediction length lower global warming p: respiratory illness
1 4
0 5
1 6
1 7
1 8
prediction length lower bloating p: indigestion
1 9
1 10
1 11
0 12
prediction length lower hull p: the hull
0 13
1 14
1 15
prediction length lower heat p: smoke
1 16
prediction length lower wind p: smoke
1 17
1 18
1 19
prediction length lower suffering p: the suffering
prediction length lower exploitation p: social injustice
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
0 20
prediction length lower suffering p: the suffering
predict

prediction length lower glare p: sunlight
1 128
1 129
prediction length lower alleged abuse p: the alleged abuse
prediction length lower alleged abuse p: the alleged abuse
prediction length lower bruises p: swelling
0 130
prediction length lower alleged abuse p: the alleged abuse
prediction length lower alleged abuse p: the alleged abuse
0 131
1 132
prediction length lower disease p: sickness
1 133
prediction length lower incoming water p: the incoming water
0 134
prediction length lower bond speculation p: the falling interest-rate structure
prediction length lower falling interest-rate structure p: the falling interest-rate structure
0 135
prediction length lower earthquake p: the earthquake
prediction length lower fire p: the fire
0 136
1 137
1 138
1 139
prediction length lower violence p: the violence
prediction length lower violence p: the death
prediction length lower violence p: the death
0 140
1 141
1 142
Cp ('The widespread opinion that  protein  and  phosphorus  cause  calciu

In [29]:
prec=sum(prediction)/len(C_p)
rec = sum(prediction)/len(C_t)
F1 = 2 *(prec * rec)/(prec + rec)
print('F1: ', F1)
print('prec: ', prec)
print('rec: ', rec)


F1:  0.5240847784200385
prec:  0.6098654708520179
rec:  0.4594594594594595


In [30]:
sum(prediction)

136

In [31]:
sum(pred_label)

122

In [32]:
len(prediction)

136

In [33]:
len(C_t)

296

In [34]:
len(pred_label)

223

In [35]:
len(C_p)

223

In [36]:
SCITE_df = pd.DataFrame({'pred_label': pred_label, 'C_p' : C_p})
SCITE_df

,pred_label,C_p
0,0,(Using solar electricity instead of convention...
1,1,(Using solar electricity instead of convention...
2,1,(Using solar electricity instead of convention...
3,1,(Using solar electricity instead of convention...
4,1,(Using solar electricity instead of convention...
...,...,...
218,1,"(Most illnesses, including colds and flu , ..."
219,0,"(Most illnesses, including colds and flu , ..."
220,0,"(Most illnesses, including colds and flu , ..."
221,0,(Every child and young person seen at Starship...


In [37]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#fewshot_semeval_CE_detection
#SCITE_df.to_csv('data/out/RAG_causality/task2/GPT3/rand_SCITE_CE_extract.csv')
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT4/zero_sem_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [38]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_


In [39]:
prec= 0.73
rec = 0.73
2*(prec*rec)/(prec+rec)

0.7299999999999999